In [1]:
#| default_exp mesh

In [2]:
#| export

import numpy as np
from typing import Iterable
import os
from copy import deepcopy
import warnings

import igl

In [3]:
import matplotlib.pyplot as plt

## Mesh data structure

In this notebook, we define a class for triangular meshes with UV maps, and show how to use it with the data from the ``basics_example`` folder.

### Define meta data

We then enter the relevant metadata - the filename, resolution in microns, and how much we want to subsample for segmentation purposes. This is important for correctly scaling mesh vertex positions when computing the mesh from image data.

In [6]:
metadata_dict = {'filename': 'basics_example/basics_example',
                 'resolution_in_microns': (1, 0.36, 0.36), # you can typically get this from the .tif metadata
                 'subsampling_factors': (1, 1/3, 1/3), # how much you subsampled your image for segmentation
                }

We now define our main class for mesh io and mesh handling. We save all meshes in as wavefront `.obj` files (see [wikipedia](https://en.wikipedia.org/wiki/Wavefront_.obj_file)). In python, we represent missing entries (such as a vertex which doesn't have a normal by `np.nan`.

**Important convention** For sanities sake, we will always store all mesh coordinates in microns. This means rescaling appropriately after calculating the mesh from the 3d segmentation.

For reading, writing, holding, and handling `.obj` meshes, we will create a `ObjMesh` class. You can think of an `.obj` mesh as a _map_ between two meshes (the mesh in 3d, and its texture coordinates in 2d), defined on a per-face basis.

In [8]:
#| export

def flatten(lst, max_depth=1000, iter_count=0):
    """
    Flatten a list of lists into a list.

    Also works with inhomogeneous lists, e.g., [[0,1],2]. The argument
    depth determines how "deep" to flatten the list, e.g. with max_depth=1:
    [[(1,0), (1,0)]] -> [(1,0), (1,0)].

    Parameters
    ----------
    lst : list
        list-of-lists.
    max_depth : int, optional
        To what depth to flatten the list.
    iter_count : int, optional
        Helper argument for recursion depth determination.
    
    Returns
    -------
    iterator
        flattened list.
    """
    for el in lst:
        if (isinstance(el, Iterable) and not isinstance(el, (str, bytes))
                and iter_count < max_depth):
            yield from flatten(el, max_depth=max_depth,
                               iter_count=iter_count+1)
        else:
            yield el
            
def pad_list(lst, length=3, fill_value=np.nan):
    """Pad end of list with fill_value if shorter than desired length."""
    return lst + max([0, (length-len(lst))]) * [fill_value,]

def unique(sequence):
    """Create list of unique entries in sequence while preserving order"""
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def index_else_nan(arr, inds):
    """Return arr[inds], masked so that the result is np.nan wherever ind is nan"""
    if len(np.shape(inds)):
        inds = np.array(inds)
    mask = np.isnan(inds)
    masked_inds = np.copy(inds)
    masked_inds[mask] = 0
    masked_inds = masked_inds.astype(int)    
    selected = arr[masked_inds]
    selected[mask] = np.nan
    return selected

def invert_dictionary(my_map, assume_unique=False):
    """
    Invert key -> value map defined by a dictionary
    
    If assume_unique is True, key/value pairs are assumed to be unique.
    Else, a dictionary of list returned. Each entry is a list
    of keys that map to the given value.
    """
    if assume_unique:
        return {v: k for k, v in my_map.items()}
    inv_map = {}
    for k, v in my_map.items():
        inv_map[v] = inv_map.get(v, []) + [k]
    return inv_map

In [9]:
#| export

class ObjMesh:
    """
    Simple class for reading, holding, transforming, and saving 3d polygonal meshes in the .obj format.
    See https://en.wikipedia.org/wiki/Wavefront_.obj_file.
    Attributes
        - vertices = [(x_0, y_0, z_0), ... ]
        - texture_vertices = [(u_0, v_0), ...] or None
        - normals = [(nx_0, ny_0, nz_0), ...] or None
        - faces = [f0, ...]
        - only_vertices = bool. 
    vertices, texture_vertices, normals are np.arrays, faces is a list.
    Each face is either a list of vertex indices (if only_vertices is True), or, if the mesh
    has texture nformation, a list of vertex/texture vertex index pairs. 
    Normals are always defined per-vertex, i.e. self.normals[i] is the normal vector at
    self.vertices[i].   Missing data is represented by np.nan.
    Faces can be any length (triangles, quads, ...). Indices start at 0!
    
    The methods self.get_uv_index_to_vertex_index_map(), self.get_uv_matched_vertex_indices()
    can be used to map data from 3d to UV, for instance for cartographic interpolation:
    self.vertices[self.get_vertex_to_texture_vertex_indices()] will give you the 3d coordinates
    at each texture vertex point in 2d.
    """
    
    def __init__(self, vertices, faces, texture_vertices=None, normals=None, name=None):
        self.vertices, self.faces = (vertices, faces)
        self.texture_vertices, self.normals = (texture_vertices, normals)
        self.name = None

    @staticmethod
    def read_obj(filename):
        """
        Return vertices, texture vertices, normals, and faces from an obj file.

        Faces are lists of pairs vertex/texture vertex. If a certain vertex has no texture 
        associated to it, the entry is np.nan, else it is an index into the vertex/texture arrays
        (note: indices of returned faces start at 0!). See https://en.wikipedia.org/wiki/Wavefront_.obj_file.
        
        Intended for .obj files containing a single object only. Will emit a warning if multiple objects
        are detected.

        Parameters
        ----------
        filename : str
            filename
        Returns
        -------
        mesh: ObjMesh
        """
        def _str_to_int_or_nan(x):
            """Convert string to int or np.nan if string is empty"""
            if x == '':
                return np.nan
            return int(x)
        with open(filename, 'r') as f:
            lines = f.readlines()
        names = [ln.split()[1:] for ln in lines if ln.startswith("o ")]
        if len(names) > 1:
            warnings.warn(f"Warning: multiple meshes in .obj file", RuntimeWarning)
        name = None if len(names) == 0 else names[0]
        vs = np.array([ln.split()[1:] for ln in lines if ln.startswith("v ")]).astype(float)
        vts = np.array([ln.split()[1:] for ln in lines if ln.startswith("vt ")]).astype(float)
        ns = np.array([ln.split()[1:] for ln in lines if ln.startswith("vn ")]).astype(float)
        fs = [ln.split()[1:] for ln in lines if ln.startswith("f ")]
        fs = [[pad_list([_str_to_int_or_nan(y)-1 for y in x.split("/")], length=3, fill_value=np.nan)
               for x in f] for f in fs]
        if vts.shape == (0,) and ns.shape == (0,): # if there is no texture information
            fs = [[v[0] for v in f] for f in fs]
            mesh = ObjMesh(vs, fs, texture_vertices=None, normals=None, name=name)
        else: 
            # match up normals to vertices
            v_n_pairs = [np.nan for v in range(vs.shape[0])]
            for v, _, n in flatten(fs, max_depth=1):
                v_n_pairs[v] = n
            ns = index_else_nan(ns, np.array(v_n_pairs))
            ns = (ns.T/np.linalg.norm(ns, axis=1)).T
            fs = [[v[:2] for v in f] for f in fs]
            mesh = ObjMesh(vs, fs, texture_vertices=vts, normals=ns, name=name)
        return mesh
        
    def write_obj(self, filename, include_uv_and_normals=True):
        """
        Write mesh to .obj format.

        Can write texture coordinates and normals if included. 

        Parameters
        ----------
        filename : str
            filename to save to
        include_uv_and_normals : bool, default True
            include uv and normal information if available.
            
        Returns
        -------
        None

        """
        def _int_or_nan_to_str(x):
            """Convert int/nan to string. np.nan is converted to empty string"""
            if np.isnan(x):
                return ''
            return str(x)
        namelines = ["o {}\n".format(self.name)] if self.name is not None else []
        if self.only_vertices:
            vlines = ["v {} {} {}\n".format(*v) for v in self.vertices]
            flines = ["f {} {} {}\n".format(*[int(v+1) for v in fc]) for fc in self.faces]
            with open(filename, 'w') as f:
                f.writelines(namelines)
                f.writelines(vlines)
                f.writelines(flines)
        if not self.only_vertices and not include_uv_and_normals:
            vlines = ["v {} {} {}\n".format(*v) for v in self.vertices]
            flines = ["f {} {} {}\n".format(*[int(v[0]+1) for v in fc]) for fc in self.faces]
            with open(filename, 'w') as f:
                f.writelines(namelines)
                f.writelines(vlines)
                f.writelines(flines)
        if not self.only_vertices and include_uv_and_normals:
            assert all([len(v)==2 for v in flatten(self.faces, max_depth=1)]), "each vertex must have 2 indices"
            texture_vertices = [] if self.texture_vertices is None else self.texture_vertices
            vlines = ["v {} {} {}\n".format(*v) for v in self.vertices]
            vtlines = ["vt {} {}\n".format(*vt) for vt in texture_vertices]
            nlines = ["vn {} {} {}\n".format(*n) for n in self.normals] if self.normals is not None else []
            faces_with_normals = [[[v[0], v[1], v[0]] for v in fc] for fc in self.faces]
            flines = ["f {} {} {}\n".format(*["{}/{}/{}".format(*[_int_or_nan_to_str(ix+1) for ix in v])
                                              for v in fc]) for fc in faces_with_normals]
            with open(filename, 'w') as f:
                f.writelines(namelines)
                f.writelines(vlines)
                f.writelines(vtlines)
                f.writelines(nlines)
                f.writelines(flines)
        return None
    
    @property
    def only_vertices(self):
        if self.texture_vertices is None:
            assert all([not isinstance(v, Iterable) for v in flatten(self.faces, max_depth=1)]), \
                "If texture_vertices is None, faces must be lists of vertex indices only"
            return True
        return False
    
    @property
    def is_triangular(self):
        """Checks if mesh has triangular faces only."""
        return all([len(fc)==3 for fc in self.faces]) 
            
    @property
    def tris(self):
        """Get all 3d triangles in mesh as a numpy array. Entries are vertex indices."""
        if self.only_vertices:
            return np.array([fc for fc in self.faces if len(fc)==3])
        return np.array([[v[0] for v in fc] for fc in self.faces if len(fc)==3])
    
    @property
    def texture_tris(self):
        """Get all texture triangles in mesh as a numpy array. Entries are texture_vertex indices."""
        if self.only_vertices:
            return np.array([np.nan for fc in self.faces if len(fc)==3])
        return np.array([[v[1] for v in fc] for fc in self.faces if len(fc)==3])
    
    @property
    def vertex_textures(self):
        """Get array of vertex texture coordinates. If multiple textures per vertex are stored,
        the last one is returned."""
        if self.only_vertices:
            return np.nan*np.ones_like(self.vertices)[:,:2]
        if len(self.texture_vertices) == 0:
            return np.nan*np.ones_like(self.vertices)[:,:2]
        v_vt_pairs = [np.nan for v in range(self.vertices.shape[0])] # in case there are stray vertices w/out face
        for v, vt in flatten(self.faces, max_depth=1):
            v_vt_pairs[v] = vt
        return index_else_nan(self.texture_vertices, np.array(v_vt_pairs))
    
    def set_normals(self):
        """Recompute normals based on 3d positions. Only works for triangular meshes."""
        if not self.is_triangular:
            warnings.warn(f"Warning: mesh not triangular - normals may be incorrect", RuntimeWarning)
        normals = igl.per_vertex_normals(self.vertices, self.tris,)
        self.normals = (normals.T / np.linalg.norm(normals, axis=1)).T
        return None
        
    def get_uv_index_to_vertex_index_map(self):
        """Get map from texture vertex index to the corresponding 3d vertex index as a dictionary."""
        return {v[1]: v[0] for v in flatten(self.faces, max_depth=1) if not np.isnan(v[1])}
    
    def get_vertex_index_to_uv_index_map(self):
        """
        Get map from 3d vertex index to the corresponding UV vertex index as a dictionary.
        
        Note: each dict item is a list, since a 3d vertex can map to multiple UV vertices.
        For vertices that do not have any correspondig UV vertex, the list is empty.
        """
        uv_to_vertex = self.get_uv_index_to_vertex_index_map()
        vertex_to_uv = invert_dictionary(uv_to_vertex, assume_unique=False)
        missing = set(range(self.vertices.shape[0])).difference(set(vertex_to_uv.keys()))
        for m in missing:
            vertex_to_uv[m] = []
        return vertex_to_uv
        
    def get_vertex_to_texture_vertex_indices(self): # to do: rename
        """
        Get an array of indices into 3d vertices that map them to the corresponding texture vertices.
        
        Useful for translating per-vertex data into per-texture-vertex data:
        array -> array[self.get_vertex_to_texture_vertex_indices()]
        """
        texture_vertex_dict = self.get_uv_index_to_vertex_index_map()
        return np.array([texture_vertex_dict[i] for i in range(self.texture_vertices.shape[0])])
        
    def map_per_vertex_to_per_texture_vertex(self, field):
        """
        Map per-vertex field (array of shape (n_vertices, ...)) to texture vertices.
        vertices.
        """
        return field[self.get_vertex_to_texture_vertex_indices()]
        
    def map_per_texture_vertex_to_per_vertex(self, field):
        """
        Map per-texture vertex field (array of shape (n_texture_vertices, ...)) to 3d vertices.
        vertices.
        """
        vertex_to_uv = self.get_vertex_index_to_uv_index_map()
        return np.stack([field[vertex_to_uv[i]].mean(axis=0) for i in range(self.vertices.shape[0])], axis=0)
        
    def cut_along_seams(self):
        """
        Cut mesh along texture seams.

        Returns a new ObjMesh in which the topology of the vertices matches the topology of the texture vertices,
        by duplicating vertices along "seams" (i.e. which have multiple corresponding texture vertices),
        and discarding any vertices without texture coordinates.
        
        Returns
        -------
        ObjMesh

        """
        assert not self.only_vertices and len(self.normals) > 0 and len(self.texture_vertices) > 0, \
            """Method requires texture or normal information"""
        texture_vertex_dict = {v[1]: v[0] for v in flatten(self.faces, max_depth=1) if not np.isnan(v[1])}
        matched_vertex_inds = np.array([texture_vertex_dict[i] for i in range(self.texture_vertices.shape[0])])
        matched_vertices = self.vertices[matched_vertex_inds]
        matched_normals = index_else_nan(self.normals, matched_vertex_inds)
        cut_faces = [[[v[1], v[1]] for v in fc] for fc in self.faces if not any([np.isnan(v[1]) for v in fc])]
        return ObjMesh(matched_vertices, cut_faces, texture_vertices=self.texture_vertices,
                       normals=matched_normals, name=self.name)
        
    def apply_affine_to_mesh(self, trafo,):
        """
        Apply affine transformation to mesh.
        
        Rotate/shear and translate vertices, rotate/shear and renormalize normals,
        flip faces if transformation determinant is -1.

        Parameters
        ----------
        trafo : np.array of shape (4,4) or (3,3)
            Transformation matrix. If (4,4), it is interpreted as affine transformation.

        Returns
        -------
        newmesh : ObjMesh
            Transformed mesh.

        """
        assert trafo.shape==(3,3) or trafo.shape==(4,4), "Transformation matrix must be 3*3 or 4*4"
        if trafo.shape == (3,3):
            trafo_matrix, trafo_translate = (trafo, np.zeros(3))
        elif trafo.shape == (4,4):
            trafo_matrix, trafo_translate = (trafo[:3,:3], trafo[:3,-1])
        newmesh = deepcopy(self)
        newmesh.vertices = self.vertices@trafo_matrix.T + trafo_translate
        if self.normals is not None:
            normals_transformed = self.normals@trafo_matrix.T
            newmesh.normals = (normals_transformed.T / np.linalg.norm(normals_transformed, axis=-1)).T
        if np.linalg.det(trafo_matrix) < 0:
            newmesh.faces = [fc[::-1] for fc in self.faces]
        return newmesh

In [10]:
#| export

def read_other_formats_without_uv(filename):
    """
    Return vertices and faces from a non-.obj mesh file format. file.

    Supported formats are: obj, off, stl, wrl, ply, mesh.
    Will NOT read in normals or texture coordinates. If you have texture
    coordinates, save your mesh as .obj. Will only return triangular faces.

    See https://libigl.github.io/libigl-python-bindings/igl_docs/#read_triangle_mesh.

    Parameters
    ----------
    filename : str
        filename
    Returns
    -------
    mesh: ObjMesh
        Only contains face and vertex info.
    """
    vs, fs = igl.read_triangle_mesh(filename)
    ns = igl.per_vertex_normals(vs, fs)
    return ObjMesh(vs, fs, texture_vertices=None, normals=ns, name=None)


### Test read/write 

In [11]:
read_other_formats_without_uv("wrapping_example/mixed_faces_example.obj")

  o Cube


In [12]:
%%time
mesh = ObjMesh.read_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv.obj")

CPU times: user 165 ms, sys: 8 ms, total: 173 ms
Wall time: 173 ms


In [13]:
mesh.vertices.shape, mesh.normals.shape, mesh.texture_vertices.shape

((8160, 3), (8160, 3), (8288, 2))

In [14]:
np.einsum('vi,vi->v', mesh.normals, igl.per_vertex_normals(mesh.vertices, mesh.tris))

array([0.99695517, 0.99865324, 0.98954003, ..., 0.99875341, 0.99958208,
              nan])

In [15]:
# test this on an example
mesh_fname_ref = "registration_example/Drosophila_reference_preregistered.obj"
mesh_fname_data = "registration_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj"

mesh = ObjMesh.read_obj(mesh_fname_ref)

## Mesh cutting and gluing

The `cut_along_seams` method can be used to "cut" a mesh along its UV seams. This duplicates 3d vertices so that the 3d topology and the unwrapped UV topololy match. For example, if your UV map cuts your sphere mesh into a north and a south hemisphere for unwrapping, `cut_along_seams` will cut the mesh into two halves.

Conversely, `glue_seams` undoes the cutting by merging close vertices.

In [16]:
#| export

def glue_seams(mesh, decimals=None):
    """
    Merge close vertices.

    Useful to undo cutting of meshes along UV seams. Note: the exact order of vertices will
    not in general be recovered by glueing after cutting.

    Parameters
    ----------
    mesh : ObjMesh
    decimals : int or None, default 10
        Vertices whose positions agree up to 'decimals' decimals are merged. Note: you can use negative values.
        If None, estimate a value based on shortest edge length in the mesh (-2*log_10(minimum length))

    Returns
    -------
    glued_mesh : ObjMesh
        Mesh with merged vertices.

    """
    if decimals is None:
        ls = igl.edge_lengths(mesh.vertices, mesh.tris)
        decimals = np.round(-2*np.log10(ls.min())).astype(int)
    rounded_verts = np.round(mesh.vertices, decimals=decimals)
    unique_verts, index, inverse_index = np.unique(rounded_verts, axis=0, return_index=True, return_inverse=True)
    sort_index = index.argsort()
    sort_sort_index = sort_index.argsort()
    unique_verts = unique_verts[sort_index]
    glued_faces = [[[sort_sort_index[inverse_index[v[0]]], v[1]] for v in fc] for fc in mesh.faces]
    glued_mesh = ObjMesh(unique_verts, glued_faces, texture_vertices=mesh.texture_vertices,
                         normals=None, name=mesh.name)
    glued_mesh.set_normals()
    return glued_mesh

In [17]:
mesh = ObjMesh.read_obj("movie_example/initial_uv.obj")

In [18]:
mesh_cut = mesh.cut_along_seams()
glued_mesh = glue_seams(mesh_cut)

In [19]:
mesh.vertices.shape, mesh_cut.vertices.shape, glued_mesh.vertices.shape

((20212, 3), (20623, 3), (12036, 3))

In [20]:
glued_mesh.write_obj("movie_example/initial_uv_glued.obj")

## Timing the read/write method

In [109]:
%%timeit
ObjMesh.read_obj(f"{metadata_dict['filename']}_mesh_marching_cubes.obj")

/tmp/ipykernel_50229/735907688.py:66: RuntimeWarning: Warning: multiple meshes in .obj file
  warnings.warn(f"Warning: multiple meshes in .obj file", RuntimeWarning)


161 ms ± 3.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [110]:
%%time
## try a somewhat larger mesh with texture info etc

mesh = ObjMesh.read_obj("registration_example/Drosophila_reference.obj")

/tmp/ipykernel_50229/735907688.py:66: RuntimeWarning: Warning: multiple meshes in .obj file
  warnings.warn(f"Warning: multiple meshes in .obj file", RuntimeWarning)


CPU times: user 1.34 s, sys: 12.1 ms, total: 1.36 s
Wall time: 1.35 s


In [111]:
%%time

mesh.write_obj("registration_example/Drosophila_reference_test_write.obj")

CPU times: user 1.15 s, sys: 19.9 ms, total: 1.17 s
Wall time: 1.16 s


In [86]:
%%time
_ = igl.read_obj("registration_example/Drosophila_reference.obj")

CPU times: user 194 ms, sys: 16.2 ms, total: 211 ms
Wall time: 207 ms


  o embryo_rect
